In [1]:
import os
import itertools

import numpy as np
from doit.tools import register_doit_as_IPython_magic
register_doit_as_IPython_magic()

from gamtools import segregation, cosegregation, matrix
from gam_follow_up import config, correlations

In [2]:
def get_matrix_task(segregation_file,
                    matrix_type='dprime', output_format='txt.gz',
                    final_dir=None):
    
    chroms = ['chr{}'.format(c) for c in range(1,20)]
    
    def internal_task_function():
    
        for chrom in chroms:

            region = [chrom]

            intermediate_output_file = cosegregation.get_output_file(
                segregation_file, region, matrix_type, output_format)
            
            if final_dir is not None:
                final_output_file = os.path.join(
                    final_dir, os.path.basename(intermediate_output_file))
            else:
                final_output_file = intermediate_output_file

            yield {
                'name': region[0],
                'targets': [final_output_file],
                'file_dep': [segregation_file],
                'actions': [(cosegregation.create_and_save_contact_matrix,
                             (segregation_file, region,
                              final_output_file, output_format, matrix_type))],
                'verbosity': 2
            }
            
    return internal_task_function

In [3]:
def split_df(input_df):
    selected_cols_1 = np.random.choice(input_df.columns, len(input_df.columns) / 2, replace=False)
    selected_cols_2 = [c for c in input_df.columns if c not in selected_cols_1]
    return input_df[selected_cols_1], input_df[selected_cols_2]

In [4]:
np.random.seed(64448)

In [5]:
segregation_base = config.in_data_processed(
        'segregation-tables/combined_segregation_at_{resolution}.table')

resolutions = ['40kb', '250kb', '1Mb']
replicates = list(range(1, 6))

In [ ]:
dict_of_tasks = {}

for replicate, resolution in itertools.product(replicates, resolutions):
    seg_table = segregation.open_segregation(segregation_base.format(resolution=resolution))
        
    seg_a, seg_b = split_df(seg_table)
    
    seg_half = {'a': seg_a, 'b': seg_b}
        
    for sample_set in ('a', 'b'):

        task_name = 'task_get_matrices_rep{}{}_at_{}_resolution'.format(
            replicate,
            sample_set,
            resolution)
        
        matrix_dir = config.in_data_processed(
            'contact-matrices/combined_replicates/{resolution}/rep{replicate}{set}').format(
                resolution=resolution,
                replicate=replicate,
                set=sample_set)

        if not os.path.exists(matrix_dir):
            os.makedirs(matrix_dir)
            
        seg_path = os.path.join(
            matrix_dir, 'segregation_at_{resolution}.table'.format(resolution=resolution))
        
        if not os.path.exists(seg_path):
            seg_half[sample_set].to_csv(seg_path, sep='\t')
    
        dict_of_tasks[task_name] = get_matrix_task(seg_path)
    
globals().update(dict_of_tasks)

In [12]:
%doit --process=5

-- get_matrices_rep5b_at_40kb_resolution:chr1
-- get_matrices_rep5b_at_40kb_resolution:chr2
-- get_matrices_rep5b_at_40kb_resolution:chr3
-- get_matrices_rep5b_at_40kb_resolution:chr4
-- get_matrices_rep5b_at_40kb_resolution:chr5
-- get_matrices_rep5b_at_40kb_resolution:chr6
-- get_matrices_rep5b_at_40kb_resolution:chr7
-- get_matrices_rep5b_at_40kb_resolution:chr8
-- get_matrices_rep5b_at_40kb_resolution:chr9
-- get_matrices_rep5b_at_40kb_resolution:chr10
-- get_matrices_rep5b_at_40kb_resolution:chr11
-- get_matrices_rep5b_at_40kb_resolution:chr12
-- get_matrices_rep5b_at_40kb_resolution:chr13
-- get_matrices_rep5b_at_40kb_resolution:chr14
-- get_matrices_rep5b_at_40kb_resolution:chr15
-- get_matrices_rep5b_at_40kb_resolution:chr16
-- get_matrices_rep5b_at_40kb_resolution:chr17
-- get_matrices_rep5b_at_40kb_resolution:chr18
-- get_matrices_rep5b_at_40kb_resolution:chr19
-- get_matrices_rep4a_at_1Mb_resolution:chr1
-- get_matrices_rep4a_at_1Mb_resolution:chr2
-- get_matrices_rep4a_at_1

In [16]:
matrix_base_path = config.in_data_processed(
    'contact-matrices/combined_replicates/{resolution}/rep{rep_num}{rep_half}/'
    'segregation_at_{resolution}.{chrom}_dprime.txt.gz')

replicate_correlations = {}
for resolution in resolutions:
    res_corrs = []
    for replicate in replicates:
        rep_corrs = []
        for chrom in correlations.chroms:
            _w, mat_a = matrix.read_file(
                matrix_base_path.format(resolution=resolution,
                                                rep_num=replicate,
                                                rep_half='a',
                                                chrom=chrom))
            _w, mat_b = matrix.read_file(
                matrix_base_path.format(resolution=resolution,
                                                rep_num=replicate,
                                                rep_half='b',
                                                chrom=chrom))

            rep_corrs.append(
                correlations.correlate_matrices(
                    correlations.keep_diagonals(mat_a, 0, 100),
                    correlations.keep_diagonals(mat_b, 0, 100)))
        res_corrs.append(rep_corrs)
    replicate_correlations[resolution] = res_corrs

In [21]:
np.array(replicate_correlations['40kb']).mean(axis=1).mean()

0.65828771877142811